### É necessário instalar os módulos Scrapy e PyMongo:
```sh
pip install scrapy
#ou 
conda install -c anaconda scrapy
```
```sh
pip install pymongo
```

O Jupyter neste caso serve só de bloco de notas. Todo o trabalho foi realizado em terminal com ajuda de um editor de texto (VSCode).

### Etapas:
1. Criar projeto
```sh
scrapy startproject "nome do projeto"
# neste caso
scrapy startproject stack
```
2. Entrar no projeto
```sh
cd "nome do projeto"
# neste caso
cd stack
```

3. Editar os arquivos
```sh
# como eu uso o VSCode
code .
```
   - Editar items.py
   
   ```py
   from scrapy import Item, Field
   
   class StackItem(Item):
       title = Field()
        url = Field()
    ```
    
    - Criar o ficheiro "stack_spider.py" dentro da pasta spider
    ```py
    class StackSpider(Spider):
    name = "stack"
    allowed_domains = ["stackoverflow.com"]
    start_urls = [
        "http://stackoverflow.com/questions?pagesize=50&sort=newest",
    ]

    def parse(self, response):
        questions = Selector(response).xpath('//div[@class="summary"]/h3')

        for question in questions:
            item = StackItem()
            item['title'] = question.xpath(
                'a[@class="question-hyperlink"]/text()').extract()[0]
            item['url'] = question.xpath(
                'a[@class="question-hyperlink"]/@href').extract()[0]
            yield item
      ```
    
    - Testar se "Spider" esta a conectar com o site
    ```sh
    scrapy crawl stack
    ```
    
    - Gravar as informações num arquivo JSON
    ```sh
    scrapy crawl stack -o itens.json -t json
    ```
    
    - Configurar o scrapy de forma a guardar as informações diretamente no MongoDB
        - Editar settings.py - adicionar
        ```py
        ITEM_PIPELINES = {'stack.pipelines.MongoDBPipeline':300}

        MONGODB_SERVER = "localhost"
        MONGODB_PORT = 27017
        MONGODB_DB = "stackoverflow"
        MONGODB_COLLECTION = "questions"
        ```
        - Editar o pipelines.py - adicionar
        ```py
        import pymongo
        from scrapy.conf import settings
        from scrapy.exceptions import DropItem
        from scrapy import log
        ```
        ```py
        class MongoDBPipeline(object):

            def __init__(self):
                connection = pymongo.MongoClient(
                    settings['MONGODB_SERVER'],
                    settings['MONGODB_PORT']
                )
                db = connection[settings['MONGODB_DB']]
                self.collection = db[settings['MONGODB_COLLECTION']]

            def process_item(self, item, spider):
                valid = True
                for data in item:
                    if not data:
                        valid = False
                        raise DropItem("Missing {0}!".format(data))
                if valid:
                    self.collection.insert(dict(item))
                    log.msg("Question added to MongoDB database!",
                            level=log.DEBUG, spider=spider)
                return item
                ```
         - Testar o Scrapy
         ```sh
         scrapy crawl stack
         ```
         
         - Visualizar o dados no MongoDB atarvés do terminal ou usar Robomongo que agora de chama Studio 3T 
         


The End